The pySPaRTAN package and the pipeline outlined in this chapter are designed around the Scanpy/scVerse ecosystem, which stores data as `AnnData` objects.  An AnnData object contains a data matrix with columns (observations) as cells and rows (variables) as genes or ADTs.  The object has dataframes `.obs` and `.var` containing information on each cell and each gene, respectively.  Additionally, the fields `.obsm` and `.varm` can be used to store additional dataframes with features describing the cells and genes.  For example, we will store the surface protein count matrix in `.obsm`.

## 3.1 Importing and pre-processing CITE-seq data

Import all the packages used in the pipeline.

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

import pySPaRTAN

Load CITE-seq data into memory using one of Scanpy’s i/o functions. In this case, the counts for mRNA and protein ADTs are stored in the same .h5 file, and so we use the `sc.read_10x_h5` function. Refer to the Scanpy documentation for other formats, such as h5ad, loom, csv, or excel files, for example.

After the data is loaded into an AnnData object, use the `var_names_make_unique` function appends a number to the end of gene names that are not unique.


In [2]:
#!wget https://cf.10xgenomics.com/samples/cell-exp/3.1.0/5k_pbmc_protein_v3_nextgem/5k_pbmc_protein_v3_nextgem_filtered_feature_bc_matrix.h5 -O ../data/cite-seq/5k_pbmc_protein_v3_nextgem_filtered_feature_bc_matrix.h5
path="../data/cite-seq/5k_pbmc_protein_v3_nextgem_filtered_feature_bc_matrix.h5"
adata=sc.read_10x_h5(path, gex_only=False)

adata.var_names_make_unique()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Separate the gene expression counts from the ADT counts. Variables are annotated as either “Antibody Capture” or “Gene Expression” in the `adata.var["feature_types"]` field. We move the proteomics data (“Antibody Capture”) to a separate data frame.

In [3]:
protein=adata[:,adata.var["feature_types"]=="Antibody Capture"].copy()
protein=protein[:,[x for x in protein.var_names if "control" not in x]]
adata.obsm["protein_raw_counts"]=protein.to_df()

adata = adata[:, adata.var["feature_types"] == "Gene Expression"]
adata.layers["raw_counts"] = adata.X.copy()


Some cells have a sequencing depth that is too low to use for our analysis.  We recommend filtering out cells where less than 5000 unique genes are sequenced, and cells where more than 30% of genes are originate from the mitochondria.  

The function `sc.pp.filter_genes` is used to remove genes that are not present in any cells, `sc.pp.calculate_qc_metrics` computes the metrics used to filter cells and stores the results in `adata.obs` and `adata.var`.  

In [4]:
sc.pp.filter_cells(adata, min_genes=1000)
sc.pp.filter_genes(adata, min_counts=0.03*adata.n_obs)
adata.var['mt'] = adata.var_names.str.startswith('MT-')

sc.pp.calculate_qc_metrics(adata, 
                           qc_vars=['mt'], 
                           percent_top=None,
                           log1p=False,
                           inplace=True)

adata=adata[adata.obs.query(
    "n_genes_by_counts < 5000 and pct_counts_mt<30").index]

In [5]:
sc.pp.highly_variable_genes(
    adata, 
    n_top_genes=5000,
    subset=True,
    flavor='seurat_v3')

adata.X=sc.pp.normalize_total(adata,
                              target_sum=10000, 
                              layer="raw_counts", 
                              inplace=False)['X']

sc.pp.log1p(adata)

adata.layers["log1p"]=normalize(
    adata.X,
    axis=1
).todense()

Trying to set attribute `.uns` of view, copying.


The protein count data  *center log ratio* (CLR) transform using the `clr` function in pySPaRTAN.  The CLR transform scales the data by the geometric mean of each cell, and then log-tranforms the data.

In [6]:
adata.obsm["protein"]=pySPaRTAN.clr(adata.obsm["protein_raw_counts"].T).T

adata.obsm["protein"].loc[:]=normalize(adata.obsm["protein"],
                                       axis=1)


## 1.2 Clustering and assigning cell-types
SPaRTAN performs best when used to model data from different cell types separately.  Several methods exist for cell-type assignments from CITE-Seq data.  For the purposes of this demonstration, we use the Leiden algorithm to cluster the data using both gene and protein expression, and then manually map the clusters to cell types based on the mean expression of the marker genes and proteins for each cluster.  The process is ommited here, but is available on the SPaRTAN Github repository.  

We use the marker genes and proteins in Table X to assign cell types to each cluster.


| Cell Type          | Marker Genes         | Marker Proteins |
|--------------------|----------------------|-----------------|
| CD8+               | CD8A,  CD8B,  FCER1G | CD8a, CD4       |
| Naive CD4+ T       | IL7R,  CCR7,  CD3E   | CD45RA, CD4     |
| Memory CD4+        | IL7R, S100A4, CD3E   | CD45RO, CD4     |
| NK                 | GNLY,  NKG7          | CD56            |
| DC                 | FCER1A, CST3         |                 |
| CD14+ Mono         | CD14, LYZ            | CD14            |
| FCGR3A+/CD16+ Mono | FCGR3A, MS4A7        | CD16            |
| B                  | MS4A1                | CD20,  CD19     |




Once the cell type assignment has been completed, and the results are stored in a csv file with two columns, one for the cells barcode, and another for its cell type, we read in the data and store it in the field `adata.obs["cell_types"]`.

In [7]:
ct_df=pd.read_csv("cell_types_PBMC.csv",index_col=0)
adata=adata[np.intersect1d(adata.obs_names, ct_df.index)]
adata.obs["cell_types"] = ct_df.loc[adata.obs_names]

Trying to set attribute `.obs` of view, copying.


## 1.3 TF-Gene binding data

A dataframe containing interactions between TFs and genes is loaded and stored in the AnnData object as follows.  The dataframe has a column for each TF and a row for each gene, with entries being either zero or one indicating if a TF and gene are recorded as interacting in the DoRethEA database.  We use an abridged version of the database, with TFs and genes being filtered out if they have too few interactions.

In [8]:
tf_gene=pySPaRTAN.load_dorthea()

We store the dataframe in our AnnData object after keeping only the genes present in both the TF-gene interaction matrix and the CITE-seq dataset.  The dataframe is stored in the AnnData object as `adata.varm["tf_gene"]`.

In [9]:
common_genes=np.intersect1d(tf_gene.index, 
                            adata.var_names)
tf_gene=tf_gene.loc[common_genes]
adata= adata[:, common_genes]
tf_gene.loc[:]=normalize(tf_gene)
adata.varm["tf_gene"]=tf_gene



# 2. Fitting SPaRTAN Models for each cell-type

To recap, after successfully completing part 1, we have an AnnData object with the following fields.

- `adata.layers["log1p"]`
- `adata.obsm["protein"]`
- `adata.varm["tf_gene"]`
- `adata.obs["cell_type"]`

The prerequisite can be verified by printing the AnnData object and seeing each field present in the output.

In [10]:
print(adata)

AnnData object with n_obs × n_vars = 4216 × 2849
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'cell_types'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg', 'log1p'
    obsm: 'protein_raw_counts', 'protein'
    varm: 'tf_gene'
    layers: 'raw_counts', 'log1p'


In order to evaluate the model's performance, we will train the model using 80% of all cells in the dataset, and reserve the remaining 20% to be used only for testing the models ability to predict gene expression based on surface protein expression.

In [11]:
adata.obs["training"]= np.random.rand(adata.n_obs) < 0.8

Because transcriptional regulation inter-cellular signaling are highly dependent on cell-types, we train SPaRTAN separately on separate cell types.  We start by demonstrating with only CD14+ monocytes.  From the entire dataset, we extract two subsets: one for training the CD14+ monocyte model and one for evaluating the model.

In [12]:
adata_cd14=adata[(adata.obs["cell_types"] == "CD14+ Mono")]

adata_cd14_training=adata[(adata.obs["cell_types"] == "CD14+ Mono")
                          & (adata.obs["training"] == True)]
adata_cd14_testing =adata[(adata.obs["cell_types"] == "CD14+ Mono") 
                          & (adata.obs["training"] == False)]

The `SPaRTAN` class in pySPaRTAN is used to perform cross validation, fit the model, make predictions, infer TF activities, and estimate the correlation between proteins and TFs.  The class has the following parameters:

- 
- 
-
...

For our demonstration, we use cross validation and grid search to find the optimal values of `lamda` between 1e-5 and 10, and `alpha` between 0 and 1.  The model is initialized as follows:

In [15]:
model_cd14= pySPaRTAN.SPaRTAN(lamda=[0.0001, 10],
                    alpha=[0,0],
                    solver="SVD")

The simplest way to fit the model is by calling the `.fit()` function and providing the AnnData object containing the training data.

In [16]:
model_cd14=model_cd14.fit(adata_cd14_training)

fold: 0
	 l: 0.0	a: 0 	 score: 0.7229577211605152
	 l: 0.0	a: 0 	 score: 0.7183586064546786
	 l: 0.001	a: 0 	 score: 0.7037402754083919
	 l: 0.005	a: 0 	 score: 0.678314399603158
	 l: 0.017	a: 0 	 score: 0.6443922098485529
	 l: 0.06	a: 0 	 score: 0.6124572315606687
	 l: 0.215	a: 0 	 score: 0.5953392148927827
	 l: 0.774	a: 0 	 score: 0.5891481085529238
	 l: 2.783	a: 0 	 score: 0.5872367674380492
	 l: 10.0	a: 0 	 score: 0.5866834370634229
fold: 1
	 l: 0.0	a: 0 	 score: 0.7064315759524724
	 l: 0.0	a: 0 	 score: 0.6812129141120755
	 l: 0.001	a: 0 	 score: 0.6483089824010977
	 l: 0.005	a: 0 	 score: 0.6148917111459842
	 l: 0.017	a: 0 	 score: 0.5824931887724564
	 l: 0.06	a: 0 	 score: 0.5447094860120332
	 l: 0.215	a: 0 	 score: 0.5191055652769221
	 l: 0.774	a: 0 	 score: 0.508887073946371
	 l: 2.783	a: 0 	 score: 0.5056019348174812
	 l: 10.0	a: 0 	 score: 0.5046350989478109
fold: 2
	 l: 0.0	a: 0 	 score: 0.7030529201973207
	 l: 0.0	a: 0 	 score: 0.6796743323765178
	 l: 0.001	a: 0 	 score: 0

Alternatively, we can fit the model without using an AnnData structure by directly providing the three matrices needed: the gene expression (Y), the protein expression (P), and the tf-gene matrix (D).

In [18]:
gene_expression=adata_cd14_training.layers["log1p"]
protein=adata_cd14_training.obsm["protein"]
tf_gene_matrix=adata_cd14_training.varm["tf_gene"]

model_cd14=model_cd14.fit(Y=gene_expression, 
                          P=protein, 
                          D=tf_gene_matrix)

We can evaluate the performance  using the `.score` function, which predicts gene expression for each cell in the training set using the protein data and compares to the true gene expression.

In [21]:
protein_test=adata_cd14_testing.obsm["protein"]

gene_expression_test=np.asarray(
    adata_cd14_testing.layers["log1p"])

model_cd14.score(P=protein_test.to_numpy(),Y=gene_expression_test)

0.46251829049725823

In [20]:
model_cd14.score(adata_cd14_testing)

0.46251829049725823

The process is repeated for each cell type in a for loop, storing the models in a dictionary.

In [22]:
models=dict()

for ct in np.unique(adata.obs["cell_types"].tolist()):
    adata_training=adata[(adata.obs["cell_types"] == ct) 
                         & (adata.obs["training"] == True)]
    adata_testing =adata[(adata.obs["cell_types"] == ct) 
                         & (adata.obs["training"] == False)]

    models[ct]= pySPaRTAN.SPaRTAN(lamda=0.001,
                        alpha=0.001,
                        solver="SVD")
    models[ct].fit(adata_training)

    print("R2 value for "+ct+" cells: \t"
          +str(models[ct].score(adata_testing)))

R2 value for B cells: 	0.5779714169846493
R2 value for CD cells: 	0.7167130225070205
R2 value for CD14+ Mono cells: 	0.42685534112870116
R2 value for CD4+ Memory T cells: 	0.4520617376369506
R2 value for CD4+ Naive T cells: 	0.4236645333693825
R2 value for CD8+ cells: 	0.5128035785529972
R2 value for FCGR3A+/CD16+ Mono cells: 	0.6296312035500657
R2 value for NK cells: 	0.504907509066246


The model for a particular cell type, say CD4+ Memory T cells for example, is stored as `model["CD4+ Memory T"]`.

# 3. Infer TF Activites

From the fitted model, TF activity is inferred from protein expression by calculating $WP^T$. TF activity for every CD14+ monocyte cell is calculated by the `get_TF_activites` function, using only the protein expression as input.

In [ ]:
tf = model_cd14.get_projD(adata_cd14.obsm["protein"])

The significance of the inferred TF activity is assessed using a permutation test.  We generate the distribution for an empirical null hypothesis by shuffling the genes for each cell (randomly permuting the rows of $Y$) a specified number of times (typically, 1000), re-fitting the model, and calculating TF activity.  For each TF in each cell, we compare the inferred TF activity to the empirical null distribution of 1000 inferred TF activities and calculate a p-value as the proportion of values in the null distribution that are higher in absolute value than the true inferred TF activity. The p-value has a minimum value inversely proportional to the number of trials (1/1000, typically). 

For each cell, P-values are corrected using Bonferroni procedure for multiple hypothesis testing, which is implemented as part of the statsmodel package (`statsmodels.stats.multitest.multipletests`).  We use an adjusted p-value of 0.15 for this example.

The permutation test is done using the `get_TF_activites` function, this time specifying the number of permutations to use for the null distribution.


In [ ]:
tf, tf_p_val = model_cd14.get_projD(
    adata_cd14.obsm["protein"],
    n_trials=100,
    verbose=True
)

In [ ]:
tf

To identify TFs that are important in CD14+ monocytes, we look for TFs that have a p value of less than 0.15 in a high proportion of cells.

In [ ]:
prop_sig=(tf_p_val<0.15).mean(axis=1)
prop_sig

The inferred TF activity and corresponding p-values are stored in the AnnData object.

In [ ]:
adata_cd14.obsm["tf_activity"]=tf.T
adata_cd14.obsm["tf_p_val"]=tf_p_val.T

Repeat the process for each cell type, and create a dataframe with the proportion of cells for each cell type where each TF is significant. 

In [ ]:
tf_ct=dict()
tf=dict()
tf_p_val=dict()
tf_ct_p_vals=dict()
tf_ct_mean=dict()
for ct in np.unique(adata.obs["cell_types"]):
    print("Calculating TF activities for "+ct+" cells")
    tf[ct], tf_p_val[ct] =models[ct].get_projD(
        adata[adata.obs["cell_types"] == ct].obsm["protein"],
        n_trials=100 
    )

    tf_ct_p_vals[ct]=(tf_p_val[ct]<0.15).mean()
    tf_ct_mean[ct]=tf[ct].mean()
    
adata.obsm["tf"]=pd.concat(tf, axis=1, join="inner").T
adata.obsm["tf_p_val"]=pd.concat(tf_p_val, axis=1, join="inner").T
tf_ct_p_vals=pd.concat(tf_ct_p_vals)
tf_ct_mean=pd.concat(tf_ct_mean)
pd.concat(tf, axis=1, join="inner")

Create a dotplot of the importance of TFs across cell-types using seaborn.  We plot only the subset of TFs that are important to the transcriptional program of at least one cell-type.

In [ ]:
tfs_to_plot=tf_ct.min().sort_values().tail(60).index
tf_ct_mean=tf_ct_mean.divide(
    np.sqrt(np.square(tf_ct_mean).sum(axis=1)),
    axis=0)

plot_df=pd.melt(tf_ct_mean.reset_index(), id_vars='index')

plot_df['p_val']=pd.melt(
    tf_ct_p_vals.reset_index(), id_vars='index')['value']
plot_df.columns=["TF", "Cell_Type", "TF_Activity", "Proportion_Significant"]

ax =sns.scatterplot(
    data=plot_df.query("Proportion_Significant>0.5"),
    y="Cell_Type", x="TF", hue="TF_Activity", size="Proportion_Significant",
    palette="PiYG",sizes=(0, 250), hue_norm=(-0.8, 0.8)
)

#ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
plt.legend(bbox_to_anchor=(0,-2),
           loc='upper left',
           borderaxespad=0)


# 4. Identifying TF-protein relationships

The matrix $W$ in the model can be interpreted as interactions between TFs and proteins.  However, due to different proteins being expressed on different scales, we instead look at the correlations between protein expressions ($P$) and the inferred TF activity ($WP^T$).  The `get_tf_protein_cor` is used to compute the correlation matrix using all cells in the training data.  The function returns a data frame with TFs as columns and proteins as rows.

In [ ]:
protein_tf=model_cd14.get_tf_protein_cor()
protein_tf.head()

Repeat the process for each cell type of interest, and store the results in a dictionary.

In [ ]:
protein_tf=dict()
for ct in np.unique(adata.obs["cell_types"]):
    protein_tf[ct]=models[ct].get_tf_protein_cor()

plot the results for the relevant TFs for each cell type ...